In [2]:
import sqlite3
import pandas as pd
import os
import geopandas as gpd
from shapely.geometry import Point
import osmnx as ox

conn = sqlite3.connect("my_database_project.db")
cur = conn.cursor()

In [2]:
# Create the table Bezirke
cur.execute('''
CREATE TABLE IF NOT EXISTS Bezirke (
    Bezirk TEXT PRIMARY KEY ,
    Breitengrad REAL,
    Längengrad REAL,
    geometry TEXT
)
''')

# Create the table Standorten_Zählstelle
cur.execute('''
CREATE TABLE IF NOT EXISTS Standorten_Zählstelle (
    Zählstelle TEXT,
    Bezirk TEXT,
    Beschreibung TEXT,
    Installationsdatum DATE
)
''')

# Create the table Messquerschnitt
cur.execute('''
CREATE TABLE IF NOT EXISTS Messquerschnitt (
    MQ_KURZNAME TEXT,
    Bezirk TEXT,
    STRASSE TEXT,
    POSITION TEXT,
    POS_DETAIL TEXT,
    INBETRIEBNAHME TEXT
)
''')

In [3]:
# Fetch administrative boundaries for Berlin Bezirke
gdf = ox.features_from_place(
    "Berlin, Germany",
    tags={"boundary": "administrative", "admin_level": "6"}
)

# Filter only relevant columns
gdf = gdf[['name', 'geometry']]

# Save the GeoDataFrame as a GeoJSON file
gdf.to_file("berlin_bezirke.geojson", driver="GeoJSON")

# List of desired Berlin Bezirke
berlin_bezirke = [
    "Charlottenburg-Wilmersdorf", "Friedrichshain-Kreuzberg", "Lichtenberg",
    "Marzahn-Hellersdorf", "Mitte", "Neukölln", "Pankow", "Reinickendorf",
    "Spandau", "Steglitz-Zehlendorf", "Tempelhof-Schöneberg", "Treptow-Köpenick"
]

# Load and process GeoJSON file
gdf_bezirke = gpd.read_file("berlin_bezirke.geojson")
gdf_bezirke = gdf_bezirke[gdf_bezirke['name'].isin(berlin_bezirke)]  # Filter Bezirke
gdf_bezirke = gdf_bezirke.dissolve(by="name")  # Dissolve by name
gdf_bezirke = gdf_bezirke.to_crs("EPSG:4326")  # Ensure CRS is WGS84
gdf_bezirke['Breitengrad'] = gdf_bezirke.geometry.centroid.y
gdf_bezirke['Längengrad'] = gdf_bezirke.geometry.centroid.x
gdf_bezirke.drop(['element','id'], axis =1, inplace=True)
# Prepare the DataFrame for SQL
df_bezirke = gdf_bezirke.reset_index()
df_bezirke['geometry'] = df_bezirke['geometry'].apply(lambda geom: geom.wkt)  # Convert geometry to WKT
df_bezirke = df_bezirke.rename(columns={'name': 'Bezirk', 'geometry': 'Geometry'})

# Insert data into the SQLite table using pandas
df_bezirke[['Bezirk', 'Breitengrad', 'Längengrad', 'Geometry']].to_sql(
    'Bezirke', conn, if_exists='replace', index=False
)

print("Bezirke data saved successfully!")


Bezirke data saved successfully!


C:\Users\user\AppData\Local\Temp\ipykernel_20632\4189669575.py:25: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_bezirke['Breitengrad'] = gdf_bezirke.geometry.centroid.y
C:\Users\user\AppData\Local\Temp\ipykernel_20632\4189669575.py:26: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_bezirke['Längengrad'] = gdf_bezirke.geometry.centroid.x
C:\Users\user\AppData\Local\Temp\ipykernel_20632\4189669575.py:30: UserWarning: Geometry column does not contain geometry.
  df_bezirke['geometry'] = df_bezirke['geometry'].apply(lambda geom: geom.wkt)  # Convert geometry to WKT


In [ ]:
#überprüfen  dass alle notwendige Informationen zu Bezirke sind in der Db gespeichert
df = pd.read_sql_query('SELECT * FROM Bezirke', conn)
df

In [4]:

excel_file_path = 'Fahrrad/gesamtdatei-stundenwerte.xlsx' 
# Load the data from Excel
if os.path.exists(excel_file_path):
    df_zaehlstellen = pd.read_excel(excel_file_path, sheet_name = 'Standortdaten')
else:
    print(f"Die Datei'{excel_file_path} wurde nicht gefunden.")
    
    
df_zaehlstellen.columns = ['Zählstelle', 'Beschreibung', 'Breitengrad', 'Längengrad', 'Installationsdatum']

# Create GeoDataFrame for Zählstellen
geometry = [Point(xy) for xy in zip(df_zaehlstellen["Längengrad"], df_zaehlstellen["Breitengrad"])]
gdf_zaehlstellen = gpd.GeoDataFrame(df_zaehlstellen, geometry=geometry, crs="EPSG:4326")

# Spatial join: Match Zählstellen with Bezirke
gdf_zaehlstellen = gpd.sjoin(gdf_zaehlstellen, gdf_bezirke, how="left", predicate="within")

# Keep only relevant columns
result = gdf_zaehlstellen[["Zählstelle", "name", 'Beschreibung', 'Installationsdatum']]
result = result.rename(columns={'name': 'Bezirk'})
#insert in the database
result.to_sql('Standorten_Zählstelle', conn, if_exists = 'append', index = False)

conn.commit()

In [ ]:
#get from db // check if the inert of Standortendaten sind richtig hinzugefügt
df = pd.read_sql_query('SELECT * FROM Standorten_Zählstelle', conn)
df

In [4]:
#if we need to store in a csv File
df_zählstelle = pd.read_sql_query('SELECT * FROM Standorten_Zählstelle', conn)
# Write DataFrame to a CSV file
df_zählstelle.to_csv('Standorten_Zählstelle.csv', index=False)

In [5]:
file_path = 'Stammdaten_Verkehrsdetektion_2022_07_20.xlsx'

if os.path.exists(file_path):
    df_Messquerschnitt = pd.read_excel(file_path)
else:
    print(f"Die Datei '{file_path}' wurde nicht gefunden.")

df_Messquerschnitt.drop(['MQ_ID15','DET_NAME_ALT','DET_NAME_NEU', 'DET_ID15','ABBAUDATUM','RICHTUNG',
       'DEINSTALLIERT', 'KOMMENTAR','annotation','SPUR'], axis = 1,inplace = True)
#wir behalten nur die notwendige Informationen
df_Messquerschnitt.columns = ['MQ_KURZNAME','STRASSE','POSITION','POS_DETAIL','BREITE_WGS84','LÄNGE_WGS84',
    'INBETRIEBNAHME']

df_Messquerschnitt = df_Messquerschnitt.drop_duplicates()
df_Messquerschnitt = df_Messquerschnitt.rename(columns={'BREITE_WGS84': 'Breitengrad','LÄNGE_WGS84':'Längengrad' })
# Create GeoDataFrame for MQ
geometry = [Point(xy) for xy in zip(df_Messquerschnitt["Breitengrad"], df_Messquerschnitt["Längengrad"])]
gdf_Messquerschnitt = gpd.GeoDataFrame(df_Messquerschnitt, geometry = geometry, crs="EPSG:4326")
#match MQ with Bezirk
gdf_Messquerschnitt = gpd.sjoin(gdf_Messquerschnitt, gdf_bezirke, how = "left", predicate = "within")
result = gdf_Messquerschnitt[['MQ_KURZNAME','name','STRASSE','POSITION','POS_DETAIL','INBETRIEBNAHME']]
result = result.rename(columns={'name': 'Bezirk'})
result.to_sql('Messquerschnitt', conn, if_exists = 'append', index = False)

conn.commit()

In [5]:
df_Messquerschnitt = pd.read_sql_query('SELECT * FROM Messquerschnitt', conn)
# Write DataFrame to a CSV file
df_Messquerschnitt.to_csv('Messquerschnitt.csv', index=False)

In [6]:
cur.execute("""
CREATE TABLE IF NOT EXISTS Messdaten_auto(
    MQ_KURZNAME TEXT NOT NULL,
    DateId DATETIME NOT NULL,
    TimeId INTEGER,
    q_pkw_mq_hr INTEGER,
    FOREIGN KEY (MQ_KURZNAME) REFERENCES Messquerschnitt(MQ_KURZNAME),
    FOREIGN KEY (DateId) REFERENCES Date_dim(DateID),
    FOREIGN KEY (TimeId) REFERENCES Time_dim(timeID)
)
""")
conn.commit()

In [ ]:
#code ohne multiprocessing the other code with multi processing is vorhanden
data = []
for year in range(2018, 2024):
    for month in range(1, 13):
        str_month = str(month).zfill(2)  # Ensure two-digit month format
        file_path = f'Auto_{year}/mq_hr_{year}_{str_month}.csv.gz'
        df = pd.read_csv(file_path,delimiter =';', compression='gzip', chunksize=10000)
        #Wir behalten nur die notwendige Spalten
        df.drop(['v_pkw_mq_hr', 'q_lkw_mq_hr', 'v_lkw_mq_hr','qualitaet', 'q_kfz_mq_hr', 'v_kfz_mq_hr'],  axis = 1, inplace = True)

        date_ids = [
            cur.execute("SELECT DateID FROM Date_dim WHERE date = ?", (date,)).fetchone()[0]
            for date in df['tag'].tolist()
            ]
        mq_ids = [
            cur.execute("SELECT mq_id FROM Messquerschnitt WHERE MQ_KURZNAME = ?", (mq_name,)).fetchone()[0]
            for mq_name in df['mq_name'].tolist()
        ]
        hour_ids = df['stunde'].tolist()
        data.extend([(mq_id, date_id, hour_id,  wert) for mq_id, date_id, hour_id,  wert in zip(mq_ids,date_ids, hour_ids,  df['q_pkw_mq_hr'])])
    
        try:
            cur.executemany("INSERT INTO Messdaten_auto (Mq_id, DateID, TimeID, q_pkw_mq_hr) VALUES (?,?,?,?)" , data)          
            conn.commit()
        except sqlite3.Error as e:
            print(f'error inserting entry :{e}')

In [3]:
df = pd.read_sql_query('SELECT * FROM Messdaten_auto', conn)
df.to_csv('Messdaten_auto.csv', index=False)
#df.head()

In [8]:
cur.execute("""
CREATE TABLE IF NOT EXISTS Messdaten_Fahrrad (
    Zählstelle TEXT NOT NULL,
    DateID DATETIME NOT NULL,
    TimeID INTEGER,
    Wert INTEGER,
    FOREIGN KEY (Zählstelle) REFERENCES Standorten_Zählstelle(Zählstelle),
    FOREIGN KEY (DateID) REFERENCES Date_dim(DateID),
    FOREIGN KEY (TimeID) REFERENCES Time_dim(TimeID)
)
""")
conn.commit()

In [4]:

excel_file_path = 'Fahrrad/gesamtdatei-stundenwerte.xlsx'

excel_file = pd.ExcelFile(excel_file_path)
sheet_names = excel_file.sheet_names

data = [] 
for item in sheet_names[3:]:
    print(item)
    df = pd.read_excel(excel_file_path, sheet_name = item,index_col = 0)
    df_reset = df.reset_index()

    df_reset.iloc[:,0]= pd.to_datetime(df_reset.iloc[:,0])
    df_reset['Date'] = df_reset.iloc[:,0].dt.strftime('%d.%m.%Y')  # Extract the date part
    df_reset['Time'] = df_reset.iloc[:,0].dt.hour  # Extract the time part

    df_reset.drop('Zählstelle        Inbetriebnahme', axis = 1, inplace = True)
    columns = ['Date', 'Time'] + [col for col in df.columns if col not in ['Date', 'Time']]

    # Reorder the DataFrame
    df_reset = df_reset[columns]
    date_ids = [
        cur.execute("SELECT DateID FROM Date_dim WHERE date = ?", (date,)).fetchone()[0]
        for date in df_reset['Date'].tolist()
    ]
    hour_ids = df_reset['Time'].tolist()
    try:
        for column in df_reset.columns[2:]:
            zaehler = column.split()[0]
            zaehler_ids = [zaehler for _ in range(len(date_ids))]
            data.extend([(zaehler_id, date_id, hour_id,  wert) for zaehler_id, date_id, hour_id,  wert in zip(zaehler_ids,date_ids, hour_ids,  df_reset[column])])

        cur.executemany("INSERT INTO Messdaten_Fahrrad (Zählstelle, DateID, TimeID, Wert) VALUES (?,?,?,?)" , data)        
        
    except Exception as e:
        print(f"Es gibt ein Problem {e}")
        
        conn.commit()

Jahresdatei 2018
Jahresdatei 2019
Jahresdatei 2020
Jahresdatei 2021
Jahresdatei 2022
Jahresdatei 2023


In [5]:
df = pd.read_sql_query('SELECT * FROM Messdaten_Fahrrad', conn)
df.tail()      

,Zählstelle,DateID,TimeID,Wert
5223321,27-RE-MAR,2191,19,18.0
5223322,27-RE-MAR,2191,20,11.0
5223323,27-RE-MAR,2191,21,10.0
5223324,27-RE-MAR,2191,22,5.0
5223325,27-RE-MAR,2191,23,4.0


In [ ]:
query = cur.execute("SELECT count(*) FROM Messdaten_Fahrrad")
print(query.fetchall())

In [ ]:

#I updated the excelfile I removed unused sheets and renamed in years_sheet the 
# 17-SZ-BRE-O  to 17-SK-BRE-O 
# 17-SZ-BRE-W to 17-SK-BRE-W
# 02-MI-AL-W to 01-MI-AL-W
# 02-PA-SE-N to 11-PA-SE-N
# I have to check nan werte /null Werte /0 Werte

In [ ]:
#to check if all the Data is well stored in the database 
df = pd.read_sql_query("SELECT * FROM Messquerschnitt where MQ_KURZNAME like '%n'", conn)
df
#es gibt mq_name vorhanden mit n am Ende, die sind nicht vorhanden in messquerchnitt Datei

In [6]:
cur.close()
conn.close()